# 07 · Inner Screen & Boundaries (Blanket-Mediated Access)

Goal: illustrate an "**inner screen**" style boundary where **internal** states update using only **blanket** variables (sensory/active), 
not the external states directly. We:

1. Simulate a nonequilibrium process that couples an external coordinate $y_t$ to internal $x_t$ through a sensory channel $s_t$ (no direct $x\leftarrow y$ access).
2. Verify with **precision-matrix diagnostics** that conditioning on the blanket reduces $I\text{–}E$ coupling.
3. Implement two internal estimators:
   - **Screened update**: $\hat x_t$ updated from past $\hat x$ and **sensory** $s_t$ only.
   - **Unscreened oracle** (for comparison): $\hat x_t$ allowed to peek at true external $y_t$.
4. Show the screened estimator can still track the relevant external influence **without** violating the boundary.

> This is a concrete, toy demonstration of how internal dynamics can be interpreted as *inference* over externals using only blanket-moderated signals.

In [ ]:
# CI parametersimport osCI = os.getenv("CI", "").lower() in ("1","true","yes")T = 20000 if CI else 60000dt = 0.002print({"CI": CI, "T": T})

In [ ]:
import numpy as np, matplotlib.pyplot as plt, pandas as pdfrom persystems.blankets import blanket_block_normsnp.set_printoptions(precision=4, suppress=True)plt.rcParams['figure.dpi'] = 120rng = np.random.default_rng(1)

## 1) A minimal screened coupling process
We simulate:
- External $y$: an OU-like process.
- Sensory $s$: a noisy function of $y$.
- Internal $x$: leaky integrator driven by **sensory** $s$ but **not** $y$ directly.

This ensures that any $x$–$y$ associations must pass through the sensory blanket.

In [ ]:
def simulate_screened(T=60000, dt=0.002, seed=1):    rng = np.random.default_rng(seed)    # Parameters    ay = 1.2; Dy = 0.6   # external OU-ish drift and diffusion    ax = 0.8; bx = 1.0   # internal decay and sensory gain    sig_s = 0.5          # sensory noise    y = 0.0; x = 0.0    Y = np.zeros(T); S = np.zeros(T); X = np.zeros(T)    for t in range(T):        # external        y = y + (-ay*y)*dt + np.sqrt(2*Dy*dt)*rng.standard_normal()        s = y + sig_s*rng.standard_normal()        # internal (screened: depends on s, not y)        x = x + (-ax*x + bx*s)*dt + 0.05*np.sqrt(dt)*rng.standard_normal()        Y[t]=y; S[t]=s; X[t]=x    return X, S, YX, S, Y = simulate_screened(T=T, dt=dt, seed=1)X.shape, S.shape, Y.shape

In [ ]:
plt.figure(figsize=(8,3.5))plt.plot(Y[::200], label='external y', alpha=0.9)plt.plot(S[::200], label='sensory s', alpha=0.7)plt.plot(X[::200], label='internal x', alpha=0.9)plt.legend(fontsize=8); plt.title('Screened coupling demo (subsampled)')plt.tight_layout(); plt.show()

## 2) Blanket diagnostics
We build a sample matrix $Z=[I|A|S|E]$ by taking:
- $I=x_t$ (internal),
- $A=\dot x_t$ (active surrogate),
- $S=s_t$ (sensory),
- $E=y_{t+1}$ (external next step).

We then compute precision-matrix blocks and the Schur-complement $K_{IE\mid AS}$.

In [ ]:
dx = (np.roll(X,-1)-X)/dty_next = np.roll(Y,-1)Z = np.stack([X[:-1], dx[:-1], S[:-1], y_next[:-1]], axis=1)norms = blanket_block_norms(Z, dims={"I":1,"A":1,"S":1,"E":1}, reg=1e-4)pd.DataFrame([{k:float(v) for k,v in norms.items()}]).T.rename(columns={0:'Frobenius'})

Heuristic check: $\lVert K_{IE\mid AS}\rVert$ should be smaller than raw $\lVert K_{IE}\rVert$ if the blanket separates internal and external when conditioned on $(A,S)$.

In [ ]:
print('K_IE       =', norms['K_IE'])print('K_IE|AS    =', norms['K_IE_cond_AS'])print('Conditioning reduced IE coupling? →', norms['K_IE_cond_AS'] <= norms['K_IE'] + 1e-9)

## 3) Inner-state estimation with and without peeking beyond the screen
We build two linear trackers for $x_t$:
- **Screened:** $\hat x_{t+1}= (1-\alpha\,dt)\hat x_t + \beta\,dt\, s_t$.
- **Oracle:**   same, plus an illicit $\gamma\,dt\, y_t$ term (violates the boundary).

We compare MSE against the true internal $x_t$; the screened tracker should perform well because the *only* driver for $x$ is sensory $s$.

In [ ]:
def track_screened(S, dt, alpha=0.8, beta=1.0):    xh = 0.0; Xh = np.zeros_like(S)    for t in range(len(S)-1):        xh = xh + (-alpha*xh + beta*S[t]) * dt        Xh[t+1]=xh    return Xhdef track_oracle(S, Y, dt, alpha=0.8, beta=1.0, gamma=0.3):    xh = 0.0; Xh = np.zeros_like(S)    for t in range(len(S)-1):        xh = xh + (-alpha*xh + beta*S[t] + gamma*Y[t]) * dt        Xh[t+1]=xh    return XhXh_scr = track_screened(S, dt)Xh_orc = track_oracle(S, Y, dt)mse_scr = float(np.mean((Xh_scr - X)**2))mse_orc = float(np.mean((Xh_orc - X)**2))print({"MSE_screened": mse_scr, "MSE_oracle": mse_orc})

In [ ]:
plt.figure(figsize=(8,3.5))plt.plot(X[::200], label='true x');plt.plot(Xh_scr[::200], label='screened x̂');plt.plot(Xh_orc[::200], label='oracle x̂', alpha=0.7);plt.legend(fontsize=8); plt.title('Internal tracking (subsampled)')plt.tight_layout(); plt.show()

### Takeaways
- The **screened** internal estimator works well because internal dynamics are *causally* driven by sensory input; there's no need to peek at $y$.
- Blanket diagnostics show reduced $I$–$E$ coupling once we condition on $(A,S)$, consistent with an **inner screen** boundary.
- This toy makes concrete how internal states can be interpreted as **inference** about externals using *only* blanket variables, aligning with the Bayesian mechanics perspective.